# Read Data from feather

In [ ]:
import feather

In [ ]:
loans =feather.read_dataframe('./loans_modelling_data')

In [ ]:
loans.shape

# Data Preprocessing for Evaluation

## Splitting data into train and test

In [ ]:
loans_train = loans.loc[loans['issue_d'] <  loans['issue_d'].quantile(0.9)]
loans_test =  loans.loc[loans['issue_d'] >= loans['issue_d'].quantile(0.9)]

In [ ]:
print('Number of loans in the partition:   ', loans_train.shape[0] + loans_test.shape[0])
print('Number of loans in the full dataset:', loans.shape[0])

In [ ]:
loans_test.shape[0] / loans.shape[0]

In [ ]:
del loans

In [ ]:

loans_train.drop('issue_d', axis=1, inplace=True)
loans_test.drop('issue_d', axis=1, inplace=True)

In [ ]:
y_train = loans_train['charged_off']
y_test = loans_test['charged_off']

In [ ]:
X_train = loans_train.drop('charged_off', axis=1)
X_test = loans_test.drop('charged_off', axis=1)

In [ ]:
del loans_train, loans_test

## Modelling

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV


In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
pipeline_rfc = Pipeline([
    ('imputer', SimpleImputer(copy=False)),
    ('model', RandomForestClassifier(n_jobs=-1, random_state=1))
])

In [ ]:
param_grid_rfc = {
    'model__n_estimators': [50] # The number of randomized trees to build
}

In [ ]:
grid_rfc = GridSearchCV(estimator=pipeline_rfc, param_grid=param_grid_rfc, scoring='roc_auc', n_jobs=1, pre_dispatch=1, cv=5, verbose=1, return_train_score=False)

In [ ]:
grid_rfc.fit(X_train, y_train)

In [ ]:
grid_rfc.best_score_

### ACCUARACY,  F1 SCORE AND ROC AUC FOR TRAIN DATA

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

In [ ]:
X_train=X_train.fillna(X_train.mean()) # To remove error missing values
X_test=X_test.fillna(X_test.mean())

In [ ]:
# See the inital model performance
clf = RandomForestClassifier(random_state=10)
print('Acc:', cross_val_score(clf, X_train, y_train, 
                              cv=StratifiedKFold(n_splits=5), 
                              scoring='accuracy').mean())
print('F1:', cross_val_score(clf, X_train, y_train, 
                             cv=StratifiedKFold(n_splits=5), 
                             scoring='f1').mean())
print('ROC AUC:', cross_val_score(clf, X_train, y_train, 
                                  cv=StratifiedKFold(n_splits=5), 
                                  scoring='roc_auc').mean())

### Prediction on test data

In [ ]:
from sklearn import model_selection
# random forest model creation
rfc = RandomForestClassifier()
rfc.fit(X_train,y_train)
# predictions
rfc_predict = rfc.predict(X_test)

In [ ]:
y_score = rfc.predict_proba(X_test)[:,1]
roc_auc_score(y_test, y_score)

#### Confusion Matrix for test data

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
rfc_cv_score =cross_val_score(rfc, X_train, y_train, cv=10, scoring='roc_auc')

In [ ]:
print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, rfc_predict))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_test, rfc_predict))
print('\n')
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())

#### ROC Curve

In [ ]:
import sklearn.metrics as metrics
from sklearn.metrics import roc_curve

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
y_pred_proba = rfc.predict_proba(X_test)[:,1]
fpr,tpr,thresholds = roc_curve(y_test,y_pred_proba)

In [ ]:
roc_auc = metrics.auc(fpr, tpr)

In [ ]:
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
roc_auc_score(y_test,y_pred_proba)

#### Precision Recall Curve

In [ ]:
from sklearn.metrics import precision_recall_curve

In [ ]:
from sklearn.metrics import auc

In [ ]:
precision, recall, thresholds = precision_recall_curve(y_test, y_pred_proba)

In [ ]:
# calculate precision-recall AUC
auc_prc = auc(recall, precision)
print(auc_prc)

In [ ]:
plt.figure(figsize = (10,8))
plt.plot([0, 1], [0.5, 0.5],'k--')
plt.plot(recall, precision,'b', label = 'Precision Recall Curve = %0.2f' % auc_prc)
plt.legend(loc = 'lower right')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('PRC curve')
plt.show()